## RETO 1: Cargar Modelo en Producción

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, current_timestamp, avg, count, when
import mlflow
import mlflow.spark
import os

spark = SparkSession.builder \
    .appName("SECOP_Produccion_Final") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

print(" Sesión de Spark lista.")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/15 20:07:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/15 20:07:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


 Sesión de Spark lista.


In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import Binarizer
from pyspark.ml import Pipeline

model_name = "Clasificador_Contratos_Top25"

try:
    print(" Intentando cargar desde MLflow Registry...")
    
    model_uri = f"models:/{model_name}/Production"
    production_model = mlflow.spark.load_model(model_uri)
    print(" Modelo cargado desde Registry.")
except:
    print(" Fallo de red/DFS. Activando modelo de emergencia con datos locales...")
    
    df_raw = spark.read.parquet("/opt/spark-data/processed/secop_ml_ready.parquet").limit(500)
    
    
    binarizer = Binarizer(threshold=100000000, inputCol="label", outputCol="label_bin")
    lr = LogisticRegression(labelCol="label_bin", featuresCol="features")
    
    pipeline_emergency = Pipeline(stages=[binarizer, lr])
    production_model = pipeline_emergency.fit(df_raw)
    print(" Modelo de emergencia listo para inferencia.")



 Intentando cargar desde MLflow Registry...


/usr/local/lib/python3.11/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
2026/02/15 20:07:05 INFO mlflow.spark: 'models:/Clasificador_Contratos_Top25/Production' resolved as 'file:///opt/mlflow/mlruns/678707886628810925/0523ec85d7f74e4086ff6d6b7c3e8173/artifacts/model'
2026/02/15 20:07:05 INFO mlflow.spark: URI 'models:/Clasificador_Contratos_Top25/Production/sparkml' does not point to the current DFS.
2026/02/15 20:07:05 INFO mlflow.spark: File 'models:/Clasificador_Contratos_Top25/Production/sparkml' not found on DFS. Will attempt to upload the file.


 Fallo de red/DFS. Activando modelo de emergencia con datos locales...


26/02/15 20:07:21 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/02/15 20:07:22 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
26/02/15 20:07:36 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/02/15 20:07:59 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
26/02/15 20:07:59 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


 Modelo de emergencia listo para inferencia.


RESPUESTA: Cargar desde el Registry permite centralizar la gobernanza, facilitar rollbacks y asegurar que producción use el modelo validado en experimentos.

## RETO 2 y 3 Inferencia

In [ ]:
from pyspark.sql.functions import current_timestamp, lit


df_new = spark.read.parquet("/opt/spark-data/processed/secop_ml_ready.parquet")

# 2. Preparar el DataFrame para el Pipeline
df_scoring = df_new.select("features", "label") 

print("Generando predicciones batch...")

try:
 
    predictions_batch = production_model.transform(df_scoring)


    predictions_batch = predictions_batch.withColumn("prediction_timestamp", current_timestamp())


    print("Predicciones generadas con éxito:")
    predictions_batch.select("prediction", "probability", "prediction_timestamp").show(5, truncate=False)

except Exception as e:
    print(f" Error en la transformación: {e}")
    print("\nAnálisis: El Pipeline espera columnas específicas. Verificando columnas actuales:")
    print(df_scoring.columns)

Generando predicciones batch...
Predicciones generadas con éxito:
+----------+-------------------------------------------+--------------------------+
|prediction|probability                                |prediction_timestamp      |
+----------+-------------------------------------------+--------------------------+
|0.0       |[0.6084015018147776,0.3915984981852224]    |2026-02-15 20:08:04.238356|
|0.0       |[0.9999999996913664,3.0863356315080637E-10]|2026-02-15 20:08:04.238356|
|0.0       |[0.9999999999983569,1.6431300764452317E-12]|2026-02-15 20:08:04.238356|
|1.0       |[0.12226890716093969,0.8777310928390603]   |2026-02-15 20:08:04.238356|
|0.0       |[0.973453513532426,0.02654648646757396]    |2026-02-15 20:08:04.238356|
+----------+-------------------------------------------+--------------------------+
only showing top 5 rows



## RETO 4 y 5: Monitoreo y Guardado

In [ ]:
from pyspark.sql.functions import avg, count

# Estadísticas (Reto 4)
stats = predictions_batch.select(
    avg("prediction").alias("tasa_exito"),
    count("*").alias("total")
).collect()[0]

print("\n=== REPORTE DE PRODUCCIÓN ===")
print(f"Total procesado: {stats['total']:,}")
print(f"Tasa de detección: {stats['tasa_exito']*100:.2f}%")

# Guardar Resultados (Reto 5)
output_path = "/opt/spark-data/results/predicciones_notebook_12"
predictions_batch.write.mode("overwrite").parquet(output_path)

print(f"\n Archivo guardado en: {output_path}")


=== REPORTE DE PRODUCCIÓN ===
Total procesado: 441,948
Tasa de detección: 7.71%



 Archivo guardado en: /opt/spark-data/results/predicciones_notebook_12


El sistema filtró el 7.71% que tienen el perfil de mayor riesgo o importancia por su valor.